In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

### Berikut adalah modelling yang kami lakukan. EDA dilakukan secara terpisah menggunakan aplikasi tableau. Akan tetapi ada beberapa EDA yang dimasukan dalam notebook

In [ ]:
# Load Up data
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def get_day(day_start):
    day_end = day_start + pd.offsets.DateOffset(hours=23)
    return pd.date_range(day_start, day_end, freq="H")

def feature_engineering(data, interpol = False):
    
    from datetime import datetime, timedelta, date
    # mendapatkan fitur yang berkaitan dengan waktu dari data datetime
    data.loc[:, 'datetime'] = pd.DatetimeIndex(data.datetime)
    data.loc[:, 'Hour'] = data.datetime.apply(lambda x: x.hour)
    data.loc[:, 'month'] = data.datetime.apply(lambda x: x.month)
    data.loc[:, 'day'] = data.datetime.apply(lambda x: x.day)
    data.loc[:, 'woy'] = data.datetime.apply(lambda x: x.week)
    data.loc[:, 'year'] = data.datetime.apply(lambda x: x.year)
    data.loc[:, 'dow'] = data.datetime.apply(lambda x: x.dayofweek)
    dt = pd.DatetimeIndex(data['datetime'])
    data.set_index(dt, inplace=True)

    # tax day
    data.loc[get_day(pd.datetime(2011, 4, 15)), "workingday"] = 1
    data.loc[get_day(pd.datetime(2012, 4, 16)), "workingday"] = 1
    # thanksgiving friday
    data.loc[get_day(pd.datetime(2011, 11, 25)), "workingday"] = 0
    data.loc[get_day(pd.datetime(2012, 11, 23)), "workingday"] = 0
    # tax day
    data.loc[get_day(pd.datetime(2011, 4, 15)), "holiday"] = 0
    data.loc[get_day(pd.datetime(2012, 4, 16)), "holiday"] = 0

    # thanksgiving friday
    data.loc[get_day(pd.datetime(2011, 11, 25)), "holiday"] = 1 
    data.loc[get_day(pd.datetime(2012, 11, 23)), "holiday"] = 1

    #storms
    data.loc[get_day(pd.datetime(2012, 5, 21)), "holiday"] = 1

    #tornado
    data.loc[get_day(pd.datetime(2012, 6, 1)), "holiday"] = 1
    
    dt = pd.DatetimeIndex(data['datetime'])
    data.set_index(dt, inplace=True)
    data['date'] = dt.date
    
    #disaster
    data['disaster']=0
    #tornado outbreak
    data.loc[data['date']==date(2011, 4, 16),"disaster"] = 1
    data.loc[data['date']==date(2011, 4, 15),"disaster"] = 1
    data.loc[data['date']==date(2011, 4, 14),"disaster"] = 1
      
    #Hurricane katie
    data.loc[data['date']==date(2011, 9, 7),"disaster"] = 1
    data.loc[data['date']==date(2011, 9, 5),"disaster"] = 1
    data.loc[data['date']==date(2011, 9, 6),"disaster"] = 1
    data.loc[data['date']==date(2011, 9, 8),"disaster"] = 1
   
    #philipe
    data.loc[data['date']==date(2011, 10, 1),"disaster"] = 1

    #Hurricane Gordon
    data.loc[data['date']==date(2012, 8, 19),"disaster"] = 1
    data.loc[data['date']==date(2012, 8, 16),"disaster"] = 1
    data.loc[data['date']==date(2012, 8, 17),"disaster"] = 1
    data.loc[data['date']==date(2012, 8, 18),"disaster"] = 1
   

   
   
    #Hurricane Irene
    data.loc[data['date']==date(2011, 8, 27),"disaster"] = 1
    data.loc[data['date']==date(2011, 8, 25),"disaster"] = 1
    data.loc[data['date']==date(2011, 8, 26),"disaster"] = 1
    data.loc[data['date']==date(2011, 8, 28),"disaster"] = 1
   
    #Hurricane Ophelia
    data.loc[data['date']==date(2011, 9, 23),"disaster"] = 1
    
    # Aggregat dari demand sepedah pada tiap season
    data = data.join(data.groupby('season')['cnt'].sum(), on='season', rsuffix='_count')
    
    
    start2011 = date(2011, 3, 13)
    end2011 = date(2011, 11, 6)
    start2012 = date(2012, 3, 11)
    end2012 = date(2012, 11, 4)
    
    # Kondisi selain winter
    data['non_winter'] = data['datetime'].apply(lambda x: int(((x>=start2011)& (x<end2011)) | ((x>=start2012)& (x<end2012))))

    data['afternoon'] = 0 
    data.loc[(data['Hour'] >= 11) & (data['Hour'] <= 15), 'afternoon'] = 1 

    # kondisi ideal saat bersepedah (subjektif)
    data.loc[ (data['temp'] >0.74) & (data['windspeed']<30/67), 'ideal'] = 1
    data['ideal'] = data['ideal'].fillna(0)

    # kondisi kelembapan tinggi
    data.loc[((data['workingday'] ==1) & (data['humidity'] >= 0.6)),  'sangat_lembap'] = 1
    data['sangat_lembap'] = data['sangat_lembap'].fillna(0)

    #christmas day and others
    data['holiday'] = data[['month', 'day', 'holiday']].apply(lambda x: (x['holiday'], 1)[x['month'] == 12 and (x['day'] in [24, 26, 31])], axis = 1)
    data['workingday'] = data[['month', 'day', 'workingday']].apply(lambda x: (x['workingday'], 0)[x['month'] == 12 and x['day'] in [24, 31]], axis = 1)

    # Membuat fitur satu hari sebelum dan sesudah liburan
    holidays = data.loc[data['holiday'] == 1, 'datetime'].unique()

    data.loc[(data['datetime'] - timedelta(days=1)).isin(holidays) |   (data['datetime'] + timedelta(days=1)).isin(holidays), 'almost_holiday'] = np.int32(1)
    data['almost_holiday'] = data['almost_holiday'].fillna(np.int32(0))

    # membuat fitur yang menunjukkan keadaan peak hour 
    con1 = (data['workingday'] ==1) & ((data['Hour'] == 8) | ((data['Hour'] == 17) | (data['Hour'] == 18)) | (data['Hour'] == 12))
    con2 = (data['workingday'] == 0) & (data['Hour']<=19) & (data['Hour']>= 10)
    data.loc[(con1 | con2), 'peak'] = 1
    data['peak'] = data['peak'].fillna(0)

    # fitur benar-benar bekerja yaitu saat working = 1 dan holiday = 0 
    data.loc[((data['holiday'] ==0) & (data['workingday'] == 1)), 'really_work'] = 1 
    data['really_work'] =  data['really_work'].fillna(0)

    return data

# load up data
def load_data(fe='sol1', interpole=False):
    train = pd.read_csv('../input/mcf-mantapu/train.csv')
    test = pd.read_csv('../input/mcf-mantapu/test.csv')
    # jika tidak ingin melaukan feature engineering
    if fe == None:    
        return train, test 
    
    else:
        # jika melakukan feature engineering
        dat =  feature_engineering(pd.concat((train, test)), interpol = interpole)
        test = dat.loc[dat['casual'].isnull()]
        train = dat.dropna()
        return train, test


In [ ]:
# memisahkan data train dengan data test
train, test = load_data(interpole=True)
X_train = train.drop(['cnt', 'casual', 'registered'], axis=1)
y_reg_train = np.log1p(train['registered'])
y_cas_train = np.log1p(train['casual'])
y_cnt_train = np.log1p(train['cnt'])

In [ ]:
# membuat data validasi
from datetime import datetime, timedelta, date

val_set = pd.DataFrame(index=train.index)
for j in [2011, 2012]:
    for i in (train.month.unique()):
        val_set.loc[train.loc[(train.date > date(j, i, 17))  & train.month == i, 'cnt'].index, 'cnt'] = train.loc[(train.date > date(j, i, 17))  & train.month == i, 'cnt']
valindex = val_set.dropna().index
val_set = train.loc[valindex]

In [ ]:
# data train dan validasi untuk feature selection dan parameter tuning
train_fc = train.drop(valindex)
X_train_fc = train_fc.drop(['cnt', 'casual', 'registered','date', 'datetime'], axis=1)
y_reg_train_fc = np.log1p(train_fc['registered'])
y_cas_train_fc = np.log1p(train_fc['casual'])
y_cnt_train_fc = np.log1p(train_fc['cnt'])

X_val = val_set.drop(['cnt', 'casual', 'registered', 'date', 'datetime'], axis=1)
y_reg_val = np.log1p(val_set['registered'])
y_cas_val = np.log1p(val_set['casual'])
y_cnt_val = np.log1p(val_set['cnt'])

In [ ]:
# Melakukan prediksi terhadap validasi
def test_on_val(model, cols):
    model.fit(X_train_fc[cols], y_reg_train_fc)
    reg = model.predict(X_val[cols])
    model.fit(X_train_fc[cols], y_cas_train_fc)
    cas = model.predict(X_val[cols])
    model.fit(X_train_fc[cols], y_cnt_train_fc)
    cnt = model.predict(X_val[cols])
    return reg, cas, cnt

# Menghitung root mean squared error. Dikarenakan data sudah dalam bentuk log(p+1)
def rmse(ypred, ytrue):
    return np.sqrt(np.mean((ytrue-ypred)**2))

# algoritma untuk mencari kombinasi fitur terbaik (secara random)
def get_best_feature(model):
    min_reg = 99999
    min_cas = 99999
    min_cnt = 99999
    for _ in range(100):
        cols = np.random.choice(X_train_fc.columns, size=16, replace=False)
        rg, cs, cn = test_on_val(model, cols)
        if rmse(rg, y_reg_val) < min_reg:
            min_reg = rmse(rg, y_reg_val)
            min_reg_cols = cols
         
        if rmse(cs, y_cas_val) < min_cas:
            min_cas = rmse(rg, y_cas_val)
            min_cas_cols = cols
        
        if rmse(cn, y_cnt_val) < min_cnt:
            min_cnt = rmse(rg, y_cas_val)
            min_cnt_cols = cols
            
    print('===========END=================')
    print(' count cols: ', min_cnt_cols, 'score: ', min_cnt)
    print(' regular cols: ', min_reg_cols, 'score: ', min_reg)
    print(' casual cols: ', min_cas_cols, 'score: ', min_cas)
    return count_cols
# algoritma untuk melakukan hyperparameter tuning

def rand_search(cols ,n, mode_name):
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import GradientBoostingRegressor

    min_rmsle = 9999

    for _ in range(n):
        if mode_name == 'rf':
            param_space = {
                'n_estimators': np.random.randint(500, 1500),
                'max_depth': np.random.randint(1, 50),
                'min_samples_split' : np.random.randint(1, 30),
                'n_jobs':-1,
                'random_state': 0}
            
        elif mode_name == 'gb':
            param_space = {
                'n_estimators': np.random.randint(1000, 2000),
                'max_depth': np.random.randint(1, 30),
                'subsample': np.random.uniform(0.4,1),
                'min_sample_leaf' : np.random.randint(1, 30),
                'learning_rate':0.01,
                'loss' : 'ls', 
                'Random_state':0}
            
        cur_par = param_space    
        print('current params: ', cur_par)
        
        if mode_name == 'rf':
            model = RandomForestRegressor(**cur_par)
        elif mode_name == 'gb':
            model = GradientBoostingRegressor(**cur_par)

        cnt, cas, reg = test_on_val(model, cols)
        pred_cnt = np.expm1(cas)+np.expm1(reg)
        pred_cnt = np.log1p(pred_cnt)
        curr_rmsle = rmse(pred_cnt, y_cnt_val)
        if curr_rmsle < min_rmsle:
            min_rmsle = curr_rmsle
            best_pars = cur_par
            
    print('best rmlse: ',min_rmsle)
    print('best par: ', cur_par)
    
    return best_pars

### Pencarian fitur serta parameter tuning memakan waktu yang sangat lama. Oleh karena itu kami tidak menyarankan untuk melakukannya lagi.  

In [ ]:
# mencari kolom terbaik 
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor

# model_rf = RandomForestRegressor()
# model_gb = GradientBoostingRegressor()
# best_cols_rf = get_best_feature(model_rf)
# best_cols_gb = get_best_feature(model_gb)

In [ ]:
# mecari paramater terbaik
# rf_cols = [
#     'weather', 'temp', 'atemp', 'windspeed',
#     'workingday', 'season', 'holiday', 'sangat_lembap',
#     'Hour', 'weekday' ,'woy','peak']
# gbm_cols = [
#     'weather', 'temp', 'atemp', 'humidity', 'windspeed',
#     'holiday', 'workingday', 'season',
#     'Hour', 'dow', 'year', 'ideal', 'cnt_count', 'disaster',
#     'almost_holiday', 'non_winter']

# best_pars_rf = rand_search(rf_cols, 100, 'rf')


# best_pars_rf = rand_search(gbm_cols, 100, 'gb')


### Hasil Prediksi terhadap validation set

In [ ]:
X_val.columns

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

rf_params = {'n_estimators': 1000, 'max_depth': 15, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}
gb_params = { 'n_estimators': 1700, 
              'max_depth': 5, 
              'random_state': 0, 
              'min_samples_leaf' : 10,
              'learning_rate': 0.01, 
              'subsample': 0.7, 
              'loss': 'ls'}
rf_cols = [
    'weather', 'temp', 'atemp', 'windspeed',
    'workingday', 'season', 'holiday', 'sangat_lembap',
    'Hour', 'weekday' ,'woy','peak']
gbm_cols = [
    'weather', 'temp', 'atemp', 'humidity', 'windspeed',
    'holiday', 'workingday', 'season',
    'Hour', 'dow', 'year', 'ideal', 'cnt_count', 'disaster',
    'almost_holiday', 'non_winter'
]
rf_model = RandomForestRegressor(**rf_params)
gbm_model = GradientBoostingRegressor(**gb_params)

# predict reg
rf_model.fit(X_train_fc[rf_cols] ,y_reg_train_fc)
gbm_model.fit(X_train_fc[gbm_cols] ,y_reg_train_fc)

reg_rf = np.expm1(rf_model.predict(X_val[rf_cols]))
reg_gb = np.expm1(gbm_model.predict(X_val[gbm_cols]))

reg_rf_wk = np.expm1(rf_model.predict(X_val.loc[X_val['workingday'] == 1, rf_cols]))
reg_rf_nw = np.expm1(rf_model.predict(X_val.loc[X_val['workingday'] == 0, rf_cols]))

reg_gb_wk = np.expm1(gbm_model.predict(X_val.loc[X_val['workingday'] == 1, gbm_cols]))
reg_gb_nw = np.expm1(gbm_model.predict(X_val.loc[X_val['workingday'] == 0, gbm_cols]))

# predict cas
rf_model.fit(X_train_fc[rf_cols] ,y_cas_train_fc)
gbm_model.fit(X_train_fc[gbm_cols] ,y_cas_train_fc)

cas_rf = np.expm1(rf_model.predict(X_val[rf_cols]))
cas_gb = np.expm1(gbm_model.predict(X_val[gbm_cols]))

cas_rf_wk = np.expm1(rf_model.predict(X_val.loc[X_val['workingday'] == 1, rf_cols]))
cas_rf_nw = np.expm1(rf_model.predict(X_val.loc[X_val['workingday'] == 0, rf_cols]))

cas_gb_wk = np.expm1(gbm_model.predict(X_val.loc[X_val['workingday'] == 1, gbm_cols]))
cas_gb_nw = np.expm1(gbm_model.predict(X_val.loc[X_val['workingday'] == 0, gbm_cols]))


rf_score = rmse(np.log1p(reg_rf+cas_rf), y_cnt_val)
gb_score = rmse(np.log1p(cas_gb+reg_gb), y_cnt_val)

rf_score_casual = rmse(np.log1p(cas_rf), y_cas_val)
gb_score_casual = rmse(np.log1p(cas_gb), y_cas_val)

rf_score_registered = rmse(np.log1p(reg_rf), y_reg_val)
gb_score_registered = rmse(np.log1p(reg_gb), y_reg_val)

# the index
wk_ind = X_val.loc[X_val['workingday'] ==1].index
nw_ind = X_val.loc[X_val['workingday'] ==0].index

rf_score_nw = rmse(np.log1p(reg_rf_nw+cas_rf_nw), y_cnt_val.loc[nw_ind])
gb_score_nw = rmse(np.log1p(reg_gb_nw+cas_gb_nw), y_cnt_val.loc[nw_ind])

rf_score_casual_nw = rmse(np.log1p(cas_rf_nw), y_cas_val.loc[nw_ind])
gb_score_casual_nw = rmse(np.log1p(cas_gb_nw), y_cas_val.loc[nw_ind])

rf_score_registered_nw = rmse(np.log1p(reg_rf_nw), y_reg_val.loc[nw_ind])
gb_score_registered_nw = rmse(np.log1p(reg_gb_nw), y_reg_val.loc[nw_ind])

rf_score_wk = rmse(np.log1p(reg_rf_wk+cas_rf_wk), y_cnt_val.loc[wk_ind])
gb_score_wk = rmse(np.log1p(reg_gb_wk+cas_gb_wk), y_cnt_val.loc[wk_ind])

rf_score_casual_wk = rmse(np.log1p(cas_rf_wk), y_cas_val.loc[wk_ind])
gb_score_casual_wk = rmse(np.log1p(cas_gb_wk), y_cas_val.loc[wk_ind])

rf_score_registered_wk = rmse(np.log1p(reg_rf_wk), y_reg_val.loc[wk_ind])
gb_score_registered_wk = rmse(np.log1p(reg_gb_wk), y_reg_val.loc[wk_ind])

print('===================Overall====================')
# Overall
print('random forest gives: ', rf_score)
print('gradient boosting gives: ', gb_score)
print('random forest registered gives: ', rf_score_registered)
print('gradient boosting registered gives: ', gb_score_registered)
print('random forest casual gives: ', rf_score_casual)
print('gradient boosting casual gives: ', gb_score_casual)
print('\n')

print('===================Working====================')
# Working
print('random forest gives: ', rf_score_wk)
print('gradient boosting gives: ', gb_score_wk)
print('random forest registered gives: ', rf_score_registered_wk)
print('gradient boosting registered gives: ', gb_score_registered_wk)
print('random forest casual gives: ', rf_score_casual_wk)
print('gradient boosting casual gives: ', gb_score_casual_wk)
print('\n')

print('===================Non Working====================')
# Non-Working
print('random forest gives: ', rf_score_nw)
print('gradient boosting gives: ', gb_score_nw)
print('random forest registered gives: ', rf_score_registered_nw)
print('gradient boosting registered gives: ', gb_score_registered_nw)
print('random forest casual gives: ', rf_score_casual_nw)
print('gradient boosting casual gives: ', gb_score_casual_nw)

In [ ]:
min_score = 1000
for _ in range(300):
    alpha = np.random.uniform(0.001, 0.99)
    beta = 1-alpha
    combined_score = rmse(alpha*np.log1p(reg_rf+cas_rf)+beta*np.log1p(cas_gb+reg_gb), y_cnt_val)
    if combined_score < min_score:
        min_score = combined_score
        print('alpha: ', alpha)
        print('score: ',combined_score)
        print('\n')

### Modelling serta Prediksi

In [ ]:
# Random Forest 

 
params = {'n_estimators': 1000, 'max_depth': 15, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}

# creating the test dataset
test_sol2 =test.copy()

# creating the models
rf_model = RandomForestRegressor(**params)
# used cols
rf_cols = [
    'weather', 'temp', 'atemp', 'windspeed',
    'workingday', 'season', 'holiday', 'sangat_lembap',
    'Hour', 'weekday' ,'woy','peak']

# fit reg
rf_model.fit(X_train[rf_cols], y_reg_train)
# predict predict reg
test_sol2['rf_registered'] = np.expm1(rf_model.predict(test[rf_cols]))

#fit cas
rf_model.fit(X_train[rf_cols], y_cas_train)
# predict cas
test_sol2['rf_casual'] = np.expm1(rf_model.predict(test[rf_cols]))
# cas+reg = cnt
test_sol2['rf_cnt'] = test_sol2['rf_casual'] + test_sol2['rf_registered']
#%%
# Gradient Booster

# creating the models
params = {'n_estimators': 1700, 
          'max_depth': 5, 
          'random_state': 0, 
          'min_samples_leaf' : 10,
          'learning_rate': 0.01, 
          'subsample': 0.7, 
          'loss': 'ls'}
            
gbm_model = GradientBoostingRegressor(**params)
gbm_cols = [
    'weather', 'temp', 'atemp', 'humidity', 'windspeed',
    'holiday', 'workingday', 'season',
    'Hour', 'dow', 'year', 'ideal', 'cnt_count', 'disaster',
    'almost_holiday', 'non_winter'
]
# fit reg
gbm_model.fit(X_train[gbm_cols], y_reg_train)
# predict reg
test_sol2['gbm_registered'] = np.expm1(gbm_model.predict(test[gbm_cols]))
# fit cas
gbm_model.fit(X_train[gbm_cols], y_cas_train)
#predict cas
test_sol2['gbm_casual'] = np.expm1(gbm_model.predict(test[gbm_cols]))
# cas+reg = cnt
test_sol2['gbm_cnt'] = test_sol2['gbm_casual'] + test_sol2['gbm_registered']

#### Korealasi antar kolom 

In [ ]:
# Random Forest Column
# correlation between column and count
train[rf_cols+['cnt']].corr()

In [ ]:
# Random Forest Column
# correlation between column and count
train[gbm_cols+['cnt']].corr()

In [ ]:

# weighted average cnt
test_sol2['cnt'] = (0.2*test_sol2['rf_cnt']) + (0.8*test_sol2['gbm_cnt'])


# DISCOUNTING
#test_sol2.loc[test_sol2['date']==date(2011, 12, 24),"cnt"] = test_sol2.loc[test_sol2['date']==date(2011, 12, 24),"cnt"]*0.6
#test_sol2.loc[test_sol2['date']==date(2011, 12, 25),"cnt"] = test_sol2.loc[test_sol2['date']==date(2011, 12, 25),"cnt"]*0.6
#test_sol2.loc[test_sol2['date']==date(2011, 12, 26),"cnt"] = test_sol2.loc[test_sol2['date']==date(2011, 12, 26),"cnt"]*0.6

#test_sol2.loc[test_sol2['date']==date(2012, 12, 24),"cnt"] = test_sol2.loc[test_sol2['date']==date(2012, 12, 24),"cnt"]*0.6

#test_sol2.loc[test_sol2['date']==date(2012, 12, 25),"cnt"] = test_sol2.loc[test_sol2['date']==date(2012, 12, 25),"cnt"]*0.6
#test_sol2.loc[test_sol2['date']==date(2012, 12, 26),"cnt"] = test_sol2.loc[test_sol2['date']==date(2012, 12, 26),"cnt"]*0.6

In [ ]:
# Export
test_sol2['count']=test_sol2['cnt']
test_sol2[['datetime', 'count']].to_csv('my_pred1.csv')

<a href="./my_pred1.csv"> Download File </a>

### Post Modelling Analysis

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(pd.concat((train['cnt'], test_sol2['cnt'])), color='m', label = 'demand training')
plt.plot(test_sol2['cnt'], color='c', label ='demand prediksi')
plt.tight_layout()
plt.title('Hasil Prediksi')
plt.xlabel('Datetime dalam jam')
plt.ylabel('Demand')
plt.legend()

In [ ]:
comb_dat = pd.concat((train, test_sol2))
plt.figure(figsize=(15,10))
plt.scatter(x=comb_dat.groupby(comb_dat.index.date)['cnt'].mean().index, y=comb_dat.groupby(comb_dat.index.date)['cnt'].mean(), color='m', label = 'demand training')
plt.scatter(x= test_sol2.groupby(test_sol2.index.date)['cnt'].mean().index, y = test_sol2.groupby(test_sol2.index.date)['cnt'].mean(), color='c', label='demand prediction')
plt.tight_layout()
plt.title('Hasil Prediksi')
plt.xlabel('Datetime dalam hari')
plt.ylabel('Demand')
plt.legend()

In [ ]:
comb_dat = pd.concat((train, test_sol2))
plt.figure(figsize=(15,10))
plt.bar(x = comb_dat.groupby(comb_dat.index.hour)['cnt'].mean().index - 0.5, height=comb_dat.groupby(comb_dat.index.hour)['cnt'].mean(), color='m', label = 'demand training',width=0.5 ,align='edge')
plt.bar(x= comb_dat.groupby(comb_dat.index.hour)['cnt'].mean().index, height= test_sol2.groupby(test_sol2.index.hour)['cnt'].mean(),  color='c', label='demand prediction',width=0.4, align='edge')
plt.tight_layout()
plt.title('Hasil Prediksi')
plt.xlabel('Jam')
plt.ylabel('Demand')
plt.legend()